### Gradient Checking

Welcome to the final assignment of this week! In this assignment, you will be implementing gradient checking. 

By the end of this notebook, you will be able to:
Implement gradient checking to verify the accuracy of your backprop implementation. 



### 1 - Packages 


In [8]:
import os
os.chdir('/app/Chapter02-Hyper-parameters-fine-Tuning/W1A3')  # change the working directory to the app folder

In [12]:
import numpy as np
from testCases import *
from public_tests import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

%load_ext autoreload
%autoreload 2


2025-10-09 09:51:26.198469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-09 09:51:26.198520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-09 09:51:26.199731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-09 09:51:26.207728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-09 09:51:27.291479: W tensorflow/comp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 2 - Problem Statement

You are part of a team working on make mobile payments avaiable globally, and are asked to build a deep learning model to detect fraud -- whenever someone makes a payment, you want to see if the payment might be fraudulent, such as if the user's account has been taken over by a hacker.

You already know that backpropagation is quite challenging to implement, and sometimes has bugs. Because this is a mission-critial application, your company's CEO wants to be really certain that your implementation of backpropagation is correct. Your CEO says, "give me proof that your backpropagation is actually working" To give this reassurance, you are going to use. "Gradient checking"/

Let's do it. 

### 3 - How does Gradient Checking work?

Backpropagation computes the gradients $\frac{\partial J}{\partial \theta}$, where $\theta$ denotes the parameters of the model. $J$ is computed using forward propagation and your loss function. 

Because forward propagation is relatively easy to implement. you are comfident you got that right, and you're almost 100% sure that you're computing the cost J correctly.  Thus, you can use your code for computng J to verify the code for computing $\frac{\partial J}{\partial \theta}$. 


Let's look back at the definition of a derivative (or gradient): 
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$


If you're not familiar with the "$\displaystyle \lim_{\varepsilon \to 0}$" notation, it's just a way of saying "when $\varepsilon$ is really, really small."

$\frac{\partial J}{\partial \theta}$ is what you want to make sure you're computing correctly.
You can compute $J(\theta + \varepsilon)$ and $J(\theta - \varepsilon)$ (in the case that $\theta$ is a real number), since you're confident your implementation for $J$ is correct.
Let's use equation (1) and a small value for $\varepsilon$ to convince your CEO that your code for computing $\frac{\partial J}{\partial \theta}$ is correct!

<a name='4'></a>
## 4 - 1-Dimensional Gradient Checking

Consider a 1D linear function $J(\theta) = \theta x$. The model contains only a single real-valued parameter $\theta$, and takes $x$ as input.

You will implement code to compute $J(.)$ and its derivative $\frac{\partial J}{\partial \theta}$. You will then use gradient checking to make sure your derivative computation for $J$ is correct. 

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center><font color='purple'><b>Figure 1</b>:1D linear model </font></center></caption>

The diagram above shows the key computation steps: First start with $x$, then evaluate the function $J(x)$ ("forward propagation"). Then compute the derivative $\frac{\partial J}{\partial \theta}$ ("backward propagation"). 

<a name='ex-1'></a>
### Exercise 1 - forward_propagation

Implement `forward propagation`. For this simple function compute $J(.)$

In [13]:
## GRADE FUNCTION: forward_propagation
def forward_propagation(x, theta):
    """
    Implement the linear forward propagation (compute J) presented in Figure1 
    
    Arguments:
    x -- a real-value input
    theta -- our paramter, a real number as well
    
    Returns:
    J -- the value of J, computed using the formula in Figure 1 J(theta) = theta * x 

    """
    ## YOUR CODE STARTS HERE
    J = theta * x
    
    ## YOUR CODE ENDS HERE
    return J


    

In [14]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))
forward_propagation_test(forward_propagation)

J = 8
 All tests passed.


### Exercise 2: backward_propagation
Now, implement the backword propagation step (derivative computation) of Figure 1. That is, compute the derivative $\frac{\partial J}{\partial \theta}$ = $x$.

In [15]:
## GRADE FUNCTION: backward_propagation
def backward_propagation(x, theta):
    """
    Implement the linear backward propagation (compute dtheta) presented in Figure 1 
    
    Arguments:
    x -- a real-value input
    theta -- our paramter, a real number as well
    
    Returns:
    dtheta -- the gradient of the cost with respect to theta

    """
    ## YOUR CODE STARTS HERE
    dtheta = x
    
    ## YOUR CODE ENDS HERE
    return dtheta

In [16]:
x, theta = 3, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))
backward_propagation_test(backward_propagation)

dtheta = 3
 All tests passed.


### Exercise 3 - gradient_check
To show that the backward propagation step is correct, you will now implement gradient checking.

**Instructions**: 
- First compute "grandapprox" using the formula above (1)
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- Then compute the gradient using backward propagation (backprop), and store the result in variable `grad`
- Finally, compute the difference betwee n `gradapprox` and `grad` using the formula:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$

You will need 3 step to compute this formula:
- 1. compute the numerator: `numerator = np.linalg.norm(grad - gradapprox)`
- 2. compute the denominator: `denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)`
- 3. compute the difference: `difference = numerator / denominator` 

if this difference is small (say less than $10^{-7}$), you can be quite confident that you have computed the gradient correctly.Otherwise, there is probably a mistake in your code for backpropagation.


In [17]:
def gradient_check(x, theta, epsilon = 1e-7, print_msg =False):
    """
    Implement the gradient checking presented in Figure 2
    
    Arguments:
    x -- a real-value input
    theta -- our parameter, a real number as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient

    """
    
    # 1. Compute gradapprox
    ## YOUR CODE STARTS HERE
    thetaplus = theta + epsilon                               # Step 1
    thetaminus = theta - epsilon                              # Step 2
    J_plus = forward_propagation(x, thetaplus)               # Step 3
    J_minus = forward_propagation(x, thetaminus)             # Step 4
    gradapprox = (J_plus - J_minus) / (2 * epsilon)          # Step 5
    
    ## YOUR CODE ENDS HERE
    
    # 2. Compute grad
    grad = backward_propagation(x, theta)                    # Step 6
    
    # 3. Compute difference (≈ 0 if backward propagation is correct)
    ## YOUR CODE STARTS HERE
    numerator = np.linalg.norm(grad - gradapprox)            # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)   # Step 2'
    difference = numerator / denominator                      # Step 3'
    
    ## YOUR CODE ENDS HERE
    
    if print_msg:
        if difference > 1e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [18]:
x, theta = 3, 4
difference = gradient_check(x, theta, print_msg=True)
gradient_check_test(gradient_check, difference)

Your backward propagation works perfectly fine! difference = 7.814075313343006e-11
 All tests passed.


Congrats, the difference is smaller than the $2*10^{-7}$ threshold!  So you can have high confidence that you've correctly computed the gradient in backward_propagation().

Now. in the more general case, you cost function J has come more than a single 1D input. When you are training a neural network, $\theta$ consists of multiple matrices W and vectors b. It is important to know how to do a gradient check with higher-dimensional inputss. Let's see how to do this.



### 5 - N-Dimensional Gradient Checking
The following figure describes the forward and backward propagation of your fraud detetion model.

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center><font color='purple'><b>Figure 2</b>: Deep neural network. LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID</font></center></caption>

Let's look at your implementation of forward and backward propagation for this model.
Below is the code provided for `forward_propagation_n` and `backward_propagation_n`. Note here that n in the nsmae implies it is for n-deimensional input.

In [33]:
## GRADE FUNCTION: forward_propagation_n

def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 2.
    
    Arguments:
    X -- training set for m examples, of shape (input size, number of examples)
    Y -- labels for m examples, of shape (output size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic costs for m examples)
    cache -- a tuple with the intermediate values  ("Z1", "A1", "W1", "b1", "Z2", "A2", "W2", "b2", "Z3", "A3", "W3", "b3")

    """
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    ## cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

    

In [65]:
## Now, let's loot at the code for backward propagation.  

## Below is the code provided for  `backward_propagation_n`. Note that the n in the name implies it is for n-dimensional inputs.

def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in Figure 2.
    
    Arguments:
    X -- training set for m examples, of shape (input size, number of examples)
    Y -- labels for m examples, of shape (output size, number of examples)
    cache -- a tuple with the intermediate values (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
                 gradients["dZ3"] = ...
                 gradients["dW3"] = ...
                 gradients["db3"] = ...
                 gradients["dA2"] = ...
                 gradients["dZ2"] = ...
                 gradients["dW2"] = ...
                 gradients["db2"] = ...
                 gradients["dA1"] = ...
                 gradients["dZ1"] = ...
                 gradients["dW1"] = ...
                 gradients["db1"] = ...

    """
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    # SIGMOID -> LINEAR
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims=True)
    dA2 = np.dot(W3.T, dZ3)
    
    # RELU -> LINEAR
    dZ2 = np.multiply(dA2, np.int64(A2 > 0 )) # just converting dz2 to a correct object.
    dW2 = 1./m * np.dot(dZ2, A1.T) 
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(W2.T, dZ2)
    
    # RELU -> LINEAR
    dZ1 = np.multiply(dA1, np.int64(A1 > 0)) # just converting dz1 to a correct object.
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims=True) 
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2, 
                    "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

If you had just implemented these functions, you might not have high confidence whether they work correctly.  So let's implement gradient checking to verify that they are correct. 

**How does gradient checking work?**.

As in Section 3 and 4, you want to compare "gradapprox" to the gradient computed by backpropagation. The formula is still:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

However, $\theta$ is not a scalar anymore. It is a dictionary called "parameters". The  function "`dictionary_to_vector()`" has been implemented for you. It converts the "parameters" dictionary into a vector called "values", obtained by reshaping all parameters (W1, b1, W2, b2, W3, b3) into vectors and concatenating them.

The inverse function is "`vector_to_dictionary`" which outputs back the "parameters" dictionary.

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center><font color='purple'><b>Figure 2</b>: dictionary_to_vector() and vector_to_dictionary(). You will need these functions in gradient_check_n()</font></center></caption>

The "gradients" dictionary has also been converted into a vector "grad" using gradients_to_vector(), so you don't need to worry about that.

Now, for every single parameter in your vector, you will apply the same procedure as for the gradient_check exercise. You will store each gradient approximation in a vector `gradapprox`. If the check goes as expected, each value in this approximation must match the real gradient values stored in the `grad` vector. 

Note that `grad` is calculated using the function `gradients_to_vector`, which uses the gradients outputs of the `backward_propagation_n` function.

### Exercise 4 - gradient_check_n

Implement the function below.

**Instructions**: Here is pseudo-code that will help you implement the gradient check.

For each i in num_parameters:
- To compute `J_plus[i]`:
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- To compute `J_minus[i]`: do the same thing with $\theta^{-}$
- Compute $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

Thus, you get a vector gradapprox, where gradapprox[i] is an approximation of the gradient with respect to `parameter_values[i]`. You can now compare this gradapprox vector to the gradients vector from backpropagation. Just like for the 1D case (Steps 1', 2', 3'), compute: 
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

**Note**: Use `np.linalg.norm` to get the norms

In [70]:
## GRADE FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7, print_msg = False):
    """
    Implement the gradient checking presented in Figure 2.
    
    Arguments:
    X -- training set for m examples, of shape (input size, number of examples)
    Y -- labels for m examples, of shape (output size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    gradients -- output of backward_propagation_n, contains gradients of the cost with respect to the parameters. 
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient

    """
    
    # Set-up variables
    parameters_values, _ = dictionary_to_vector(parameters) # theta 
    grad = gradients_to_vector(gradients)                  # dtheta
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # Compute gradapprox
    for i in range(num_parameters):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        ## YOUR CODE STARTS HERE
        thetaplus = np.copy(parameters_values)              # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon        # Step 2
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus)) # Step 3
        
        ## YOUR CODE ENDS HERE
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        ## YOUR CODE STARTS HERE
        thetaminus = np.copy(parameters_values)             # Step 1'
        thetaminus[i][0] = thetaminus[i][0] - epsilon      # Step 2'
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus)) # Step 3'
        ## YOUR CODE ENDS HERE
        # Compute gradapprox[i]
        ## YOUR CODE STARTS HERE
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)  # Step 4'
        ## YOUR CODE ENDS HERE



    ## compare gradapprox to backward propagation gradients by computing difference.
    ## YOUR CODE STARTS HERE
    numerator = np.linalg.norm(grad - gradapprox)            # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)   # Step 2'
    difference = numerator / denominator                      # Step 3'
    ## YOUR CODE ENDS HERE  
    
    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")  
            
    return difference 

In [71]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y, 1e-7, True)

expected_values = [0.2850931567761623, 1.1890913024229996e-07]
assert not(type(difference) == np.ndarray), "You are not using np.linalg.norm for numerator or denominator"
assert np.any(np.isclose(difference, expected_values)), "Wrong value. It is not one of the expected values"

Your backward propagation works perfectly fine! difference = 1.186249751058462e-07


<font color='blue'>


**What you should remember from this notebook**:
- Gradient checking verifies closeness between the gradients from backpropagation and the numerical approximation of the gradient (computed using forward propagation).
- Gradient checking is slow, so you don't want to run it in every iteration of training. You would usually run it only to make sure your code is correct, then turn it off and use backprop for the actual learning process. 